In [1]:
import os

import numpy as np
from models.attention import multihead_model
from models.selfattention import selfattention_model, selfattention_model_modified
from models.baseline_model import baseline,baseline_preprocess, baseline_residual, baseline_modified
from keras.callbacks import ModelCheckpoint, EarlyStopping
from data_utility.file_utility import FileUtility
from data_utility.labeling_utility import LabelingData
import itertools
from data_utility.feedgenerator import train_batch_generator_408, validation_batch_generator_408


os.environ["CUDA_VISIBLE_DEVICES"] = '1'

Using TensorFlow backend.


In [2]:
def run_baseline(epochs=10, setting_name='basemodel', gpu='1', train_batch_size=64,
                 test_batch_size=100, patience=10, features_to_use=['onehot', 'sequence_profile'], convs=[3, 5, 7], dense_size=200, lstm_size=400,use_CRF=False,filter_size=256):
    os.environ["CUDA_VISIBLE_DEVICES"] = gpu

    # read files
    train_file = '../DeepSeq2Sec/data/s8_all_features/train.txt'
    test_file = '../DeepSeq2Sec/data/s8_all_features/test.txt'
    LD = LabelingData(train_file, test_file)
    train_lengths = [int(j) for j in FileUtility.load_list('/'.join(train_file.split('/')[0:-1]) + '/train_length.txt')]
    test_lengths = [int(i) for i in FileUtility.load_list('/'.join(test_file.split('/')[0:-1]) + '/test_length.txt')]

    # model
    model, params = baseline_preprocess(LD.n_classes, features_to_use=features_to_use, convs=convs,
                             dense_size=dense_size, lstm_size=lstm_size,use_CRF=use_CRF, filter_size=filter_size)

    # output directory
    FileUtility.ensure_dir('baseline_modifications/' + setting_name + params + '/')

    # save model
    with open('baseline_modifications/' + setting_name + params + '/' + 'config.txt', 'w') as fh:
        model.summary(print_fn=lambda x: fh.write(x + '\n'))

    # check points
    filepath = 'baseline_modifications/' + setting_name + params + "/weights-improvement-{epoch:02d}-{weighted_acc:.3f}-{val_weighted_acc:.3f}.hdf5"
    checkpoint = ModelCheckpoint(filepath, monitor='val_weighted_acc', verbose=1, save_best_only=True, mode='max',
                                 period=1)
    earlystopping = EarlyStopping(monitor='val_weighted_acc', min_delta=0, patience=patience, verbose=0, mode='max',
                                  baseline=None)
    callbacks_list = [checkpoint, earlystopping]

    # calculate the sizes
    steps_per_epoch = len(train_lengths) / train_batch_size if len(train_lengths) % train_batch_size == 0 else int(
        len(train_lengths) / train_batch_size) + 1
    validation_steps = int(len(test_lengths) / test_batch_size) if len(test_lengths) % test_batch_size == 0 else int(
        len(test_lengths) / test_batch_size) + 1

    # feed model
    h = model.fit_generator(train_batch_generator_408(train_batch_size), steps_per_epoch=steps_per_epoch,
                            validation_data=validation_batch_generator_408(test_batch_size),
                            validation_steps=validation_steps,
                            shuffle=False, epochs=epochs, verbose=1, callbacks=callbacks_list)

    # save the history
    FileUtility.save_obj('baseline_modifications/' + setting_name + params + '/history', h.history)
    return model


In [3]:
model=run_baseline(epochs=1000, setting_name='profilesig_onehotembed_', gpu='0', train_batch_size=64,
                 test_batch_size=100, patience=100, features_to_use=['onehot', 'sequence_profile'], convs=[3,5,7,11,21], dense_size=1000, lstm_size=1000,use_CRF=False, filter_size=256)

Labeling utility object created..
Training y encoded shape is  (5534, 700)
Maximum sequence length is 700
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, None, 408)    0                                            
__________________________________________________________________________________________________
onehot (Lambda)                 (None, None, 21)     0           input_1[0][0]                    
__________________________________________________________________________________________________
sequenceprofile (Lambda)        (None, None, 21)     0           input_1[0][0]                    
__________________________________________________________________________________________________
batch_normalization_3 (BatchNor (None, None, 21)     84          onehot[0][0]                     
___

Epoch 7/1000
87/87 [==============================] - 24s 274ms/step - loss: 0.7518 - weighted_acc: 0.7396 - val_loss: 1.0791 - val_weighted_acc: 0.6290

Epoch 00007: val_weighted_acc did not improve from 0.63728
Epoch 8/1000
87/87 [==============================] - 24s 273ms/step - loss: 0.7338 - weighted_acc: 0.7450 - val_loss: 1.0990 - val_weighted_acc: 0.6301

Epoch 00008: val_weighted_acc did not improve from 0.63728
Epoch 9/1000
87/87 [==============================] - 24s 274ms/step - loss: 0.7294 - weighted_acc: 0.7473 - val_loss: 1.0006 - val_weighted_acc: 0.6619

Epoch 00009: val_weighted_acc improved from 0.63728 to 0.66187, saving model to baseline_modifications/profilesig_onehotembed_model#onehot#sequence_profile@conv3_5_7_11_21@dense_1000@lstm1000@droplstm0.5@filtersize_256/weights-improvement-09-0.748-0.662.hdf5
Epoch 10/1000
87/87 [==============================] - 24s 274ms/step - loss: 0.6976 - weighted_acc: 0.7571 - val_loss: 1.0206 - val_weighted_acc: 0.6532

Epoch 


Epoch 00041: val_weighted_acc did not improve from 0.68422
Epoch 42/1000
87/87 [==============================] - 24s 271ms/step - loss: 0.2619 - weighted_acc: 0.9154 - val_loss: 1.3289 - val_weighted_acc: 0.6381

Epoch 00042: val_weighted_acc did not improve from 0.68422
Epoch 43/1000
87/87 [==============================] - 24s 271ms/step - loss: 0.2547 - weighted_acc: 0.9183 - val_loss: 1.3627 - val_weighted_acc: 0.6324

Epoch 00043: val_weighted_acc did not improve from 0.68422
Epoch 44/1000
87/87 [==============================] - 24s 273ms/step - loss: 0.2532 - weighted_acc: 0.9191 - val_loss: 1.3907 - val_weighted_acc: 0.6433

Epoch 00044: val_weighted_acc did not improve from 0.68422
Epoch 45/1000
87/87 [==============================] - 24s 271ms/step - loss: 0.2528 - weighted_acc: 0.9196 - val_loss: 1.3460 - val_weighted_acc: 0.6425

Epoch 00045: val_weighted_acc did not improve from 0.68422
Epoch 46/1000
87/87 [==============================] - 24s 271ms/step - loss: 0.2495

87/87 [==============================] - 24s 273ms/step - loss: 0.1464 - weighted_acc: 0.9598 - val_loss: 1.6958 - val_weighted_acc: 0.6620

Epoch 00080: val_weighted_acc did not improve from 0.68422
Epoch 81/1000
87/87 [==============================] - 24s 273ms/step - loss: 0.1426 - weighted_acc: 0.9609 - val_loss: 1.7441 - val_weighted_acc: 0.6675

Epoch 00081: val_weighted_acc did not improve from 0.68422
Epoch 82/1000
87/87 [==============================] - 24s 273ms/step - loss: 0.1389 - weighted_acc: 0.9620 - val_loss: 1.7548 - val_weighted_acc: 0.6651

Epoch 00082: val_weighted_acc did not improve from 0.68422
Epoch 83/1000
87/87 [==============================] - 24s 271ms/step - loss: 0.1373 - weighted_acc: 0.9627 - val_loss: 1.7598 - val_weighted_acc: 0.6661

Epoch 00083: val_weighted_acc did not improve from 0.68422
Epoch 84/1000
87/87 [==============================] - 24s 273ms/step - loss: 0.1337 - weighted_acc: 0.9639 - val_loss: 1.6882 - val_weighted_acc: 0.6651

Epo

In [ ]:
model=run_baseline(epochs=1000, setting_name='baseline_no_cnn', gpu='1', train_batch_size=64,
                 test_batch_size=100, patience=50, features_to_use=['onehot', 'sequence_profile'], convs=[3,5,7,11,21], dense_size=200, lstm_size=200,use_CRF=False, filter_size=60)